In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY = input("Developer Key를 입력하세요: ")
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# 1. 채널 id 가져오기

In [2]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

search_response = youtube.search().list(
    q = "레오제이",           # q : 검색어
    order = "relevance",    # order : 정렬방법 ex)date, rating, relevance
    part = "snippet",       # part : 필수 매개변수
    maxResults = 50         # maxResults : 결과개수
    ).execute()

channel_id = search_response['items'][0]['id']['channelId']

# 2. 모든 동영상 리스트 추출하기

In [3]:
channel = youtube.channels().list(
    id=channel_id,
    part='contentDetails'
    ).execute()

playlist_id = channel['items'][0]['contentDetails']['relatedPlaylists']['uploads']

upload_channel = youtube.channels().list(
    id=channel_id,
    part='snippet'
    ).execute()

channel_title = upload_channel['items'][0]['snippet']['title']

videos = []
next_page_token = None


# max result가 50이므로 동영상 리스트를 더 추출하기 위한 작업

while 1:
    channel = youtube.playlistItems().list(
        playlistId=playlist_id,
        part='snippet',
        maxResults=50,
        pageToken=next_page_token
    ).execute()

    videos += channel['items']
    next_page_token = channel.get('nextPageToken')

    if next_page_token is None:
        break

In [4]:
video_ids = []
for video in videos:
    ids = video['snippet']['resourceId']['videoId']
    video_ids.append(ids)

title = []
views = []
likes = []
dislikes = []
comments = []
date = []

for i in range(len(video_ids)):
    request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=video_ids[i])
    response = request.execute()

    if response['items'] == []:
        title.append('-')
        views.append('-')
        likes.append('-')
        dislikes.append('-')
        comments.append('-')

    else:
        title.append(response['items'][0]['snippet']['title'])
        date.append(response['items'][0]['snippet']['publishedAt'].split('T')[0])
        views.append(response['items'][0]['statistics']['viewCount'])
        likes.append(response['items'][0]['statistics']['likeCount'])
        dislikes.append(response['items'][0]['statistics']['dislikeCount'])
        comments.append(response['items'][0]['statistics']['commentCount'])


In [5]:
import pandas as pd
import numpy as np

list_df = pd.DataFrame([title, date, views, likes, dislikes, comments]).T
list_df.columns = ['제목', '최초 업로드일', '조회수', '좋아요', '싫어요', '댓글수']
list_df.index = np.arange(1, len(list_df) + 1)

In [6]:
import openpyxl
list_df.to_excel('./all_video_list.xlsx', sheet_name='youtube')

In [8]:
list_df.head()

,제목,최초 업로드일,조회수,좋아요,싫어요,댓글수
1,신선하다 이 조합…🪐 “1일 1숏츠” 를 위해 방구서...,2021-11-19,183908,5415,30,181
2,"이래서 안키워 내가…😣 고양이가 좋아, 강아지...",2021-11-18,240653,5505,41,366
3,"어딜뺏겨 내꺼를~!😤 ""광희""님 모셔서 배우 메이크업 해드리다 밥그릇?싸움 까아쥐…",2021-11-13,891916,23076,128,1247
4,"야 적당히해라…😭 내돈내산 ""인생 호캉스""에서 말 많 은 겟레디윗미⚠️",2021-11-11,168672,3485,35,293
5,"마! 이게 K-하이틴 이다🇰🇷 ""진절미""님 얼.굴.소.멸 하이틴으로 대변신 시켜드리기",2021-11-06,372577,7264,54,283


In [9]:
list_df.tail()

,제목,최초 업로드일,조회수,좋아요,싫어요,댓글수
253,미스 레오그린의 글램퍼플 마녀 할로윈 메이크업,2016-10-18,24565,581,19,111
254,섹시주의🚫 현실판 조커 메이크업,2016-08-30,489282,6848,374,722
255,★터틀닌자★ 터프한 닌자거북이 히어로 메이크업,2016-05-31,10355,367,5,76
256,레오제이! 노래불러줘요~ ★시아준수★ 노래 메들리 !!,2016-02-27,33243,1260,18,290
257,제시제이 X 레오제이 Flashlight ★듀엣★ 스페셜,2015-12-03,107120,3317,70,482
